# Master notebook for evaluating  performaces


In [ ]:
import os
import glob
import tqdm
import numpy as np
import mplhep as hep
import awkward as ak
import matplotlib.pyplot as plt
from hydra import compose, initialize
from omegaconf import OmegaConf

from enreg.tools import general as g
from enreg.tools.metrics import (
    regression_evaluator as re,
    decay_mode_evaluator as dme,
    tagger_evaluator as te
)

with initialize(version_base=None, config_path="../enreg/config/", job_name="test_app"):
    cfg = compose(config_name="benchmarking")

hep.style.use(hep.styles.CMS)

In [ ]:
OUTPUT_DIR = "/home/hardiveski/ml-tau-en-reg/z_temp_output/"

In [ ]:
# Import training files

# Training files base directory
BASE_DATA_DIR = "/scratch/persistent/laurits/ml-tau/20241002_Training_ntuples_geq20gev"

# Data directory
SAMPLE_DIR = {
    "z_test": os.path.join(BASE_DATA_DIR, "z_test.parquet"),
    "zh_test": os.path.join(BASE_DATA_DIR, "zh_test.parquet"),
    "qq_test": os.path.join(BASE_DATA_DIR, "qq_test.parquet"),
    "z_train": os.path.join(BASE_DATA_DIR, "z_train.parquet"),
    "zh_train": os.path.join(BASE_DATA_DIR, "zh_train.parquet"),
    "qq_train": os.path.join(BASE_DATA_DIR, "qq_train.parquet"),
}

def train_data_loader(key):
    """
    Returns ak.Array
    """
    return g.load_all_data(SAMPLE_DIR[key])

# z_test_data = train_data_loader("z_test")
zh_test_data = train_data_loader("zh_test")
# qq_test_data = train_data_loader("qq_test")

In [ ]:
def create_reco_entries(base_dir, key,  output_dir):
    """
    Creates reconstruced stuffs
    """
    data = ak.from_parquet(base_dir, columns=['reco_jet_p4s', "gen_jet_tau_p4s"])
    data = ak.Array({k: data[k] for k in data.fields})
    data_to_save = {
        "reco_jet_pt": g.reinitialize_p4(data.reco_jet_p4s).pt,
        "gen_tau_pt": g.reinitialize_p4(data.gen_jet_tau_p4s).pt
    }
    output_dir = os.path.join(output_dir, "recoJet")
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{key}.parquet")
    print(f"Saving to {output_path}")
    ak.to_parquet(ak.Record(data_to_save), output_path, row_group_size=1024)

for idx, (key, value) in enumerate(SAMPLE_DIR.items()):
    if idx == 2:
        break
    else:
        create_reco_entries(SAMPLE_DIR[key], key, OUTPUT_DIR)

In [ ]:
# Training itself
BASE_MODEL_DIR = '/home/laurits/ml-tau-en-reg/training-outputs/20201204_ParT_ntrain_v2'

# Model types
version = ["v1", "v2", "v3"]
train_frac = ["trainfrac_1e4", "trainfrac_1e5", "trainfrac_1e6", "trainfrac_2e3"]
tasks = ['jet_regression', 'dm_multiclass', 'tagger', 'binary_classification']
models = ["ParticleTransformer", "LorentzNet", "DeepSet", "HPS",
          "OmniParT_from_scratch", "OmniParT_fine_tuning", "OmniParT_fixed_backbone"]
model_test_type = ["z_test", "zh_test"]

def model_loader(base_dir, v, fraction, task, model, model_test_type):
    full_directory = [os.path.join(base_dir, v, fraction, task, model, model_test_type + '.parquet')]
    return g.load_all_data(full_directory)
                                  
PT_data = model_loader(BASE_MODEL_DIR, version[0], train_frac[0], tasks[0], 'ParticleTransformer', model_test_type[1])

In [ ]:
PT_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/ParticleTransformer/zh_test.parquet")
LN_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/LorentzNet/zh_test.parquet")
DS_data = g.load_all_data("/home/laurits/ml-tau-en-reg/training-outputs/20240921_recoPtCut_removed_samples/v1/jet_regression/DeepSet/zh_test.parquet")
hps_data = g.load_all_data("/home/laurits/HPS_recoCut0_ntuples/zh.parquet")
recoJet_data = g.load_all_data("/home/laurits/ntuples/20240924_lowered_recoPtCut/recoJet/zh.parquet")

In [ ]:
hpsPlus_mask = hps_data.true_decay_mode == hps_data.pred_decay_mode
cfg.metrics.regression.ratio_plot.resolution_plot.ylim = [0, 0.2]
cfg.metrics.regression.ratio_plot.response_plot.ylim = [0.96, 1.04]

In [ ]:
evaluators = {
    "HPS":  re.RegressionEvaluator(hps_data.pred_pt, hps_data.true_pt, cfg.metrics.regression, "zh", "HPS"),
    "HPS_":  re.RegressionEvaluator(hps_data.pred_pt[hpsPlus_mask], hps_data.true_pt[hpsPlus_mask], cfg.metrics.regression, "zh", "HPS_"),
    "RecoJet":  re.RegressionEvaluator(recoJet_data.reco_jet_pt, recoJet_data.gen_tau_pt, cfg.metrics.regression, "zh", "RecoJet"),
    "PT":   re.RegressionEvaluator(PT_data.jet_regression.pred, PT_data.jet_regression.target, cfg.metrics.regression, "zh", "ParticleTransformer"),
    "LN":  re.RegressionEvaluator(LN_data.jet_regression.pred, LN_data.jet_regression.target, cfg.metrics.regression, "zh", "LorentzNet"),
    "DS":  re.RegressionEvaluator(DS_data.jet_regression.pred, DS_data.jet_regression.target, cfg.metrics.regression, "zh", "DeepSet")
}

In [ ]:
output_dir = "/home/hardiveski/tmp/20240923_recoPtCut_removed_plots"
os.makedirs(output_dir, exist_ok=True)
rme = re.RegressionMultiEvaluator(output_dir, cfg.metrics.regression, "zh")

rme.combine_results([evaluator for evaluator in evaluators.values()])
rme.save()


In [ ]:
rme.resolution_lineplot.fig

In [ ]:
rme.response_lineplot.fig

## Decay mode performances

In [ ]:
# data = {sample: g.load_all_data(os.path.join(cfg.base_ntuple_path, sample + ".parquet")) for sample in cfg.comparison_samples}

for algorithm, algo_info in cfg.metrics.dm_reconstruction.algorithms.items():
    for signal_sample in cfg.metrics.dm_reconstruction.signal_samples:
        sig_info_data = data[signal_sample]
        if not os.path.exists(os.path.join(algo_info.data_dir, signal_sample + ".parquet")):
            continue
        sig_data = g.load_all_data(os.path.join(algo_info.data_dir, signal_sample + ".parquet"))

        output_dir = "output_plots_dm"
        evaluator = dme.DecayModeEvaluator(sig_data.dm_multiclass.pred, sig_data.dm_multiclass.target, output_dir, signal_sample, algorithm)
        evaluator.save_performance()